## 1. Building the topology
The initial topology in GNS3 is as follows:

![Alt text](images/00_initial_topology.png)

### 1.1. Configure CSR1
- The CSR1 using Cisco IOS XE version 16.7.1.
- Base configuration:
```bash
conf t
hostname A-R
ip domain-name comet.abc
vrf definition MGMT
add ipv4
end
## set int f1 as MGMT interface
conf t
int g1
vrf forwarding MGMT
ip address 192.168.122.71 255.255.255.0
end
## add the default root, name-server
conf t
ip route vrf MGMT 0.0.0.0 0.0.0.0 192.168.122.1
ip domain-lookup
ip name-server vrf MGMT 192.168.122.1
end
```

- Check the connection:
```bash
ping vrf MGMT 192.168.122.1
ping vrf MGMT cisco.com
```
- Configure ssh:
```bash
conf t
crypto key generate rsa mod 2048
username admin privilege 15 password admin
line vty 0 15
transport input all
login local
end
```

- Enable netconf:
```bash
conf t
netconf-yang
```
- Check the netconf process:
```bash
sh platform software yang-management process
```
- Turn of `service config`:
```bash
conf t
no service config
```

### 1.2. Configure network-automation container

- Check ssh connection from network-automation container:
```bash
ssh admin@192.168.122.71 # password: admin
```

### 1.3. Using ncclient to get the interface list
The details of the router A_R is as follows:
- ip address: 192.168.10.138
- netconf port: 830 (as [ref.](https://developer.cisco.com/docs/ios-xe/#!enabling-netconf-on-ios-xe/addressing-and-ports))
- username: admin
- password: admin

Another IOS-XE always on sandbox online:
```python
IOS_XE_1 = {
    "host": "ios-xe-mgmt.cisco.com",
    "username": "developer",
    "password": "C1sco12345",
    "netconf_port": 10000,
    "restconf_port": 9443,
    "ssh_port": 8181
    }
```

USING SSH with the command:

`ssh -s admin@192.168.100.146 -p 830 netconf`

In [2]:
from ncclient import manager
import xmltodict
import xml.dom.minidom


# Create an XML filter for targeted NETCONF queries
netconf_filter = """
<filter>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces>
</filter>"""

print("Opening NETCONF Connection to {}"
      .format("host: ATN-IPS-R-192.168.100.144"))

# Open a connection to the network device using ncclient
with manager.connect(
        host="192.168.100.146",
        port=830,
        username="admin",
        password="admin",
        hostkey_verify=False
        ) as m:

    print("Sending a <get-config> operation to the device.\n")
    # Make a NETCONF <get-config> query using the filter
    netconf_reply = m.get_config(source = 'startup', filter = netconf_filter)

print("Here is the raw XML data returned from the device.\n")
# Print out the raw XML that returned
print(xml.dom.minidom.parseString(netconf_reply.xml).toprettyxml())
print("")

# Parse the returned XML to an Ordered Dictionary
netconf_data = xmltodict.parse(netconf_reply.xml)["rpc-reply"]["data"]

# Create a list of interfaces
interfaces = netconf_data["interfaces"]["interface"]

print("The interface status of the device is: ")
# Loop over interfaces and report status
for interface in interfaces:
    print("Interface {} enabled status is {}".format(
            interface["name"],
            interface["enabled"]
            )
        )
print("\n")

Opening NETCONF Connection to host: ATN-IPS-R-192.168.100.144
Sending a <get-config> operation to the device.



RPCError: Unsupported capability :startup

### 1.4. Using Advance Netconf Explorer
- The software is available in the following [link](https://github.com/cisco-ie/anx).
- To install the software:
```bash
git clone https://github.com/cisco-ie/anx
cd anx/
docker build -t netconf-explorer .
docker run --name netconf-exlorer -d -p 9269:8080 netconf-explorer
```
- To run on web browser: http://localhost:9269/

